## Install Dependencies Required

In [20]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib


In [1]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import random
import numpy as np

np.random.seed(42)
TRAIN = True

The dataset is structured in the following manner:

```
├── data.yaml
├── README.dataset.txt
├── README.roboflow.txt
├── test
│   ├── images
│   └── labels
├── train
│   ├── images
│   └── labels
└── valid
    ├── images
    └── labels

```

## Helper function for setting the directory

In [2]:
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR

## Function for Monitoring Training

In [3]:
def monitor_tensorboard():
    %load_ext tensorboard
    %tensorboard --logdir runs/train --host localhost

## Clone YOLOV5 Repository

In [4]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

In [5]:
%cd yolov5/
!cd

C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\yolov5
C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\yolov5


In [6]:
!pip install -r requirements.txt

In [16]:
monitor_tensorboard()



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 19080), started 1:15:45 ago. (Use '!kill 19080' to kill it.)

In [8]:
from yolov5.train import run
RES_DIR = set_res_dir()
run(data='../data.yaml' , weights='yolov5s.pt', img=640, epochs=3, batch_size=16 , name=RES_DIR, optimizer = 'AdamW')
# wandb will ask if you want to visualize your results
# type 3 and enter to get pass it (wandb is deprecated)



wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=../data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=runs\train, name=results_9, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False, img=640


Current number of result directories: 8
results_9


github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-287-g574331f9 Python-3.11.5 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
Comet: run 'pip install comet_ml' to automatically track and visualize YOLOv5  runs in Comet
TensorBoard: Start with 'tensorboard --logdir runs\train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1          

Namespace(weights='yolov5s.pt', cfg='', data='../data.yaml', hyp={'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=WindowsPath('data/hyps'), resume_evolve=None, bucket='', cache=None, image_weights=False, device='', multi_scale=False, single_cls=False, optimizer='AdamW', sync_bn=False, workers=8, project='runs\\train', name='results_9', exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_per

## Helper Functions for inference and extracting frame number

Here we make use of the run function in yolov5's detect.py 

In [28]:
from detect import run

def inference(RES_DIR, data_path):
    # Directory to store inference results.
    infer_dir_count = len(glob.glob('runs\\detect\\*'))
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)
    # Run inference on images and videos
    run(weights = f"runs\\train\\{RES_DIR}\\weights\\best.pt", source = data_path, name = INFER_DIR, save_txt = True, save_conf = True, conf_thres=0.3)
    return INFER_DIR

In [21]:
def extract_frame_numbers_and_coordinates(directory):
    frame_coordinates_map = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.txt'):
            file_path = os.path.join(directory, file_name)
            frame_number = int(file_name.split('_')[1].split('.')[0])
            coordinates = []

            # Read lines from YOLOv5 text file
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Extract XY coordinates
            for line in lines:
                data = line.strip().split()
                x_center, y_center, width, height = map(float, data[1:5])
                
                # Calculate the coordinates from normalised bounding box to absolute pixel coordinates
                x1 = int((x_center - 0.5 * width) * 960)
                y1 = int((y_center - 0.5 * height) * 720)
                coordinates.append((x1, y1))

            frame_coordinates_map[frame_number] = coordinates

    print(frame_coordinates_map)


## Run Inference on Images collected from the video

In [29]:
IMAGE_INFER_DIR = inference(RES_DIR, '..\\inferenceImages')

YOLOv5  v7.0-287-g574331f9 Python-3.11.5 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)

Fusing layers... 


inference_23


Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_000384_PNG.rf.90c8fb3c3c5ceb8f5f1c3629f9139e12.jpg: 480x640 1 employee_tag, 9.0ms
image 2/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_000384_PNG.rf.b54b66488063d1a537f8eb8a0b4c20ad.jpg: 640x640 1 employee_tag, 8.0ms
image 3/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_000431_PNG.rf.ce2b91714e1e519f1f6a07ef7ac05c59.jpg: 640x640 1 employee_tag, 7.0ms
image 4/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_000431_PNG.rf.fbc67dc7fa7e30d8b862897e508650c7.jpg: 480x640 1 employee_tag, 7.0ms
image 5/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_000460_PNG.rf.6eb0d82b0b14cfee7545b6983aca9045.jpg: 480x640 1 employee_tag, 6.0ms
image 6/75 C:\Users\yewyn\Documents\footfal

image 47/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_001012_PNG.rf.4189bd1f3dba753518749eff946d231b.jpg: 640x640 1 employee_tag, 7.0ms
image 48/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_001015_PNG.rf.85ae535387bab19fb2a1f66b60ded973.jpg: 640x640 1 employee_tag, 6.0ms
image 49/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_001015_PNG.rf.fc8100f7c45f1ccdc3588c44c8444708.jpg: 480x640 1 employee_tag, 7.0ms
image 50/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_001022_PNG.rf.3cf9c5638081ebfb4b55dd2f8adb0e54.jpg: 640x640 1 employee_tag, 7.0ms
image 51/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_001022_PNG.rf.a697430e863980038a40cd13595714b6.jpg: 480x640 1 employee_tag, 8.0ms
image 52/75 C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\inferenceImages\frame_001025_PNG.rf.

## Run Inference on the initial Video

In [24]:
VIDEO_INFER_DIR = inference(RES_DIR, '..\\sample.mp4')

YOLOv5  v7.0-287-g574331f9 Python-3.11.5 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)

Fusing layers... 


inference_21


Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (2/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (3/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (4/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (5/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (6/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (7/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (8/1341) C:\Users\yewyn\Documents\footfall_task\tr

video 1/1 (67/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (68/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (69/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (70/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (71/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (72/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (73/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (74/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms


video 1/1 (133/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (134/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (135/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (136/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (137/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (138/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (139/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (140/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections)

...

video 1/1 (1055/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1056/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1057/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1058/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1059/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1060/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1061/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1062/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no det

video 1/1 (1120/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1121/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1122/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 9.0ms
video 1/1 (1123/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1124/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 9.0ms
video 1/1 (1125/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1126/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1127/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no det

video 1/1 (1185/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (1186/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1187/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1188/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1189/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1190/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 9.0ms
video 1/1 (1191/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1192/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no det

video 1/1 (1250/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1251/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 6.0ms
video 1/1 (1252/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1253/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1254/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1255/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1256/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 7.0ms
video 1/1 (1257/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no det

video 1/1 (1315/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1316/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1317/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 9.0ms
video 1/1 (1318/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 9.0ms
video 1/1 (1319/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1320/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1321/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no detections), 8.0ms
video 1/1 (1322/1341) C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\sample.mp4: 480x640 (no det

## Frame and XY Extraction

Here we use the function we defined earlier to get and store the frame numbers based on the txt files outputed during inference and the xy coordinates into a dictionary form to print out

In [15]:
# print frames and coordinates where the tag is detected
extract_frame_numbers_and_coordinates(f'.\\runs\\detect\\{VIDEO_INFER_DIR}\\labels')

{}
